In [ ]:
!pip install transformers datasets torch evaluate
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import json

# Veri seti JSON dosyasından yüklenir
with open('qa_final.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

contexts = []
questions = []
answers = []

for item in data:
    context = item['context']
    for qa in item['qas']:
        question = qa['question']
        answer = qa['answers'][0]
        contexts.append(context)
        questions.append(question)
        answers.append(answer)

In [ ]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

model_name='dbmdz/bert-base-turkish-cased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Veriyi işleme fonksiyonu
def preprocess_data(examples):
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        answer_start = answers['answer_start']
        answer_text = answers['text']

        start_char = answer_start
        end_char = start_char + len(answer_text)

        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1

        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions

    return tokenized_examples

# Veri Ön İşleme
# Veri seti train ve test setlerine ayrılır
data = {
    'context': contexts,
    'question': questions,
    'answers': answers
}

dataset = Dataset.from_dict(data)
train_dataset, eval_dataset = train_test_split(dataset, test_size=0.2) # train = 0.8; test = 0.2;

train_dataset = Dataset.from_dict(train_dataset)
eval_dataset = Dataset.from_dict(eval_dataset)

# Veri seti tokenizasyonu
train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=['context', 'question', 'answers'])
eval_dataset = eval_dataset.map(preprocess_data, batched=True, remove_columns=['context', 'question', 'answers'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1327 [00:00<?, ? examples/s]

Map:   0%|          | 0/332 [00:00<?, ? examples/s]

In [ ]:
# Training the model
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    eval_steps=144,
    logging_steps=144,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.235500,1.861119
2,1.646400,1.631090
3,1.119000,1.613929
4,0.872600,1.663144


TrainOutput(global_step=696, training_loss=1.543141343127722, metrics={'train_runtime': 194.6714, 'train_samples_per_second': 28.602, 'train_steps_per_second': 3.575, 'total_flos': 1091175256129536.0, 'train_loss': 1.543141343127722, 'epoch': 4.0})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1719879274.a283267bfa30.315.1:   0%|          | 0.00/6.99k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cozfuttu/results/commit/8d349e5d16db181a89fa7558aa5c7b2e397238ad', commit_message='End of training', commit_description='', oid='8d349e5d16db181a89fa7558aa5c7b2e397238ad', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

# Eğitilmiş modeli yükleyin
model_path = "./results/checkpoint-500"
qa_pipeline = pipeline("question-answering", model=model_path, tokenizer="dbmdz/bert-base-turkish-cased")

# Örnek bir soru sorun
context = "Birleşik Krallık Başbakanı David Lloyd George'a göre Yunanistan büyümeli ve İngiltere ile menfaatleri birleştirilmeliydi. Yunanistan boğazları Avrupa'ya açık tutmalı, Akdeniz'de İngiltere'nin çıkarlarına uygun davranmalıydı. Eğer böyle davranmazsa İngiliz donanması onu uslandırmak için yeterdi. Sevr Antlaşması'nın kuvvet kullanılmadan uygulanamayacağı anlaşılmıştı. İtilaf Devletleri ise kuvvet kullanacak hâlde değildi. İtilaf Devletleri, Yunanları yalnız Türk illerini alıp kendi vatanına katmak için değil kendi davalarını da yürütmek için Anadolu'ya çıkardı. Ancak İtilaf Devletleri de Türkiye'ye karşı uygulanacak politikalarda artık beraber değildir. İtalya, Yunanların Anadolu'ya yerleşmesinden dolayı rahatsızdı. Fransa ise Suriye'deki toprak kazançlarını yeterli görmektedir. Artık Yunanlar kendi ordularıyla Anadolu'ya boyun eğdirmek zorundadır. Mustafa Kemal de Yunan ordusunu yenerse, Türkiye'yi kurtarmış olacaktır. 6 Ocak 1921 günü Bursa'dan Eskişehir'e ve Uşak'tan Afyon'a doğru iki kol hâlinde ileri harekâta başlayan Yunan ordusu, 9 Ocak'ta İnönü mevzilerine kadar ilerledi. Ancak Türk ordusunun savunması karşısında ileri gidemeyeceklerini anlayarak 11 Ocak 1921 sabahı İnönü mevzilerinden çekilmek zorunda kaldı. Birkaç gün sonra geride kalan Çerkez Ethem birlikleri milli birlikler tarafından dağıtıldı. Birinci İnönü Muharebesi düzenli ordunun ilk zaferi olduğundan Kuvâ-yi Milliye'den düzenli orduya geçiş hızlanmış, halkın yeni kurulan orduya güveni artmıştır. Bu başarı bütün dünyanın dikkatini çekmiş; İtilaf Devletleri, 26 Ocak 1921'de Osmanlı Devleti'nin Londra'ya bir heyet göndermesini ve bu toplantıda Ankara Hükûmeti'nden de temsilci bulundurulmasını istemişlerdir. 1 Mart'ta Albay İsmet tuğgeneral rütbesine terfi etti."
question = "Yunan ordusu, İnönü mevzilerine ne zaman ilerledi?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.4386031925678253, 'start': 1050, 'end': 1056, 'answer': '9 Ocak'}
